# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 17 2022 11:35AM,239845,20,ACI-7938570-FLS-BO-FLS,"ACI Healthcare USA, Inc.",Released
1,Jun 17 2022 11:35AM,239845,20,ACI-8040355-FL,"ACI Healthcare USA, Inc.",Released
2,Jun 17 2022 11:35AM,239845,20,7942955-BO1,"ACI Healthcare USA, Inc.",Released
3,Jun 17 2022 11:35AM,239845,20,ACI-7942967-BO1-FLS,"ACI Healthcare USA, Inc.",Released
4,Jun 17 2022 11:35AM,239845,20,7996654-2,"ACI Healthcare USA, Inc.",Released
5,Jun 17 2022 11:35AM,239845,20,8113722-FL,"ACI Healthcare USA, Inc.",Released
6,Jun 17 2022 11:35AM,239845,20,8113730-FL,"ACI Healthcare USA, Inc.",Released
7,Jun 17 2022 11:35AM,239845,20,8113731,"ACI Healthcare USA, Inc.",Released
8,Jun 17 2022 11:35AM,239845,20,7878924-2,"ACI Healthcare USA, Inc.",Released
9,Jun 17 2022 11:35AM,239845,20,8108675-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,239842,Released,1
31,239843,Released,1
32,239844,Released,1
33,239845,Released,52
34,239846,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239842,NaN,NaN,1.0
239843,NaN,NaN,1.0
239844,NaN,NaN,1.0
239845,NaN,NaN,52.0
239846,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239762,0.0,0.0,1.0
239763,0.0,0.0,1.0
239775,11.0,4.0,0.0
239781,0.0,0.0,1.0
239786,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239762,0,0,1
239763,0,0,1
239775,11,4,0
239781,0,0,1
239786,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239762,0,0,1
1,239763,0,0,1
2,239775,11,4,0
3,239781,0,0,1
4,239786,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239762,,,1
1,239763,,,1
2,239775,11,4,
3,239781,,,1
4,239786,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 17 2022 11:35AM,239845,20,"ACI Healthcare USA, Inc."
52,Jun 17 2022 11:31AM,239846,16,"SHL Pharma, LLC"
53,Jun 17 2022 11:20AM,239844,18,"ClearSpec, LLC"
54,Jun 17 2022 11:17AM,239843,10,Emerginnova
55,Jun 17 2022 11:06AM,239842,10,"Citieffe, Inc."
56,Jun 17 2022 10:51AM,239840,19,ACG North America LLC
59,Jun 17 2022 10:24AM,239838,10,Eywa Pharma Inc.
60,Jun 17 2022 10:18AM,239837,10,"Citieffe, Inc."
61,Jun 17 2022 10:16AM,239836,19,"Bi-Coastal Pharma International, LLC"
62,Jun 17 2022 10:09AM,239835,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 17 2022 11:35AM,239845,20,"ACI Healthcare USA, Inc.",,,52
1,Jun 17 2022 11:31AM,239846,16,"SHL Pharma, LLC",,,1
2,Jun 17 2022 11:20AM,239844,18,"ClearSpec, LLC",,,1
3,Jun 17 2022 11:17AM,239843,10,Emerginnova,,,1
4,Jun 17 2022 11:06AM,239842,10,"Citieffe, Inc.",,,1
5,Jun 17 2022 10:51AM,239840,19,ACG North America LLC,,,3
6,Jun 17 2022 10:24AM,239838,10,Eywa Pharma Inc.,,,1
7,Jun 17 2022 10:18AM,239837,10,"Citieffe, Inc.",,,1
8,Jun 17 2022 10:16AM,239836,19,"Bi-Coastal Pharma International, LLC",,,1
9,Jun 17 2022 10:09AM,239835,10,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 11:35AM,239845,20,"ACI Healthcare USA, Inc.",52,,
1,Jun 17 2022 11:31AM,239846,16,"SHL Pharma, LLC",1,,
2,Jun 17 2022 11:20AM,239844,18,"ClearSpec, LLC",1,,
3,Jun 17 2022 11:17AM,239843,10,Emerginnova,1,,
4,Jun 17 2022 11:06AM,239842,10,"Citieffe, Inc.",1,,
5,Jun 17 2022 10:51AM,239840,19,ACG North America LLC,3,,
6,Jun 17 2022 10:24AM,239838,10,Eywa Pharma Inc.,1,,
7,Jun 17 2022 10:18AM,239837,10,"Citieffe, Inc.",1,,
8,Jun 17 2022 10:16AM,239836,19,"Bi-Coastal Pharma International, LLC",1,,
9,Jun 17 2022 10:09AM,239835,10,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 11:35AM,239845,20,"ACI Healthcare USA, Inc.",52,,
1,Jun 17 2022 11:31AM,239846,16,"SHL Pharma, LLC",1,,
2,Jun 17 2022 11:20AM,239844,18,"ClearSpec, LLC",1,,
3,Jun 17 2022 11:17AM,239843,10,Emerginnova,1,,
4,Jun 17 2022 11:06AM,239842,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 11:35AM,239845,20,"ACI Healthcare USA, Inc.",52.0,NaN,NaN
1,Jun 17 2022 11:31AM,239846,16,"SHL Pharma, LLC",1.0,NaN,NaN
2,Jun 17 2022 11:20AM,239844,18,"ClearSpec, LLC",1.0,NaN,NaN
3,Jun 17 2022 11:17AM,239843,10,Emerginnova,1.0,NaN,NaN
4,Jun 17 2022 11:06AM,239842,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 11:35AM,239845,20,"ACI Healthcare USA, Inc.",52.0,0.0,0.0
1,Jun 17 2022 11:31AM,239846,16,"SHL Pharma, LLC",1.0,0.0,0.0
2,Jun 17 2022 11:20AM,239844,18,"ClearSpec, LLC",1.0,0.0,0.0
3,Jun 17 2022 11:17AM,239843,10,Emerginnova,1.0,0.0,0.0
4,Jun 17 2022 11:06AM,239842,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2398200,14.0,13.0,5.0
12,479647,1.0,3.0,0.0
15,479632,21.0,9.0,5.0
16,959176,4.0,0.0,0.0
18,239844,1.0,0.0,0.0
19,959316,12.0,23.0,14.0
20,959274,66.0,10.0,11.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2398200,14.0,13.0,5.0
1,12,479647,1.0,3.0,0.0
2,15,479632,21.0,9.0,5.0
3,16,959176,4.0,0.0,0.0
4,18,239844,1.0,0.0,0.0
5,19,959316,12.0,23.0,14.0
6,20,959274,66.0,10.0,11.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,14.0,13.0,5.0
1,12,1.0,3.0,0.0
2,15,21.0,9.0,5.0
3,16,4.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,12.0,23.0,14.0
6,20,66.0,10.0,11.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,14.0
1,12,Released,1.0
2,15,Released,21.0
3,16,Released,4.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Completed,5.0,0.0,5.0,0.0,0.0,14.0,11.0
Executing,13.0,3.0,9.0,0.0,0.0,23.0,10.0
Released,14.0,1.0,21.0,4.0,1.0,12.0,66.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Completed,5.0,0.0,5.0,0.0,0.0,14.0,11.0
1,Executing,13.0,3.0,9.0,0.0,0.0,23.0,10.0
2,Released,14.0,1.0,21.0,4.0,1.0,12.0,66.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Completed,5.0,0.0,5.0,0.0,0.0,14.0,11.0
1,Executing,13.0,3.0,9.0,0.0,0.0,23.0,10.0
2,Released,14.0,1.0,21.0,4.0,1.0,12.0,66.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()